In [1]:
import pandas as pd
import numpy as np

In [2]:
#import datasets
features = pd.read_csv('Inputs/CSV/features.csv')
sales = pd.read_csv('Inputs/CSV/sales.csv')
stores = pd.read_csv('Inputs/CSV/stores.csv')

In [3]:
#align date column
features['Date'] = pd.to_datetime(features['Date'])
sales['Date'] = pd.to_datetime(sales['Date'])

In [4]:
#merge dataframes
total = pd.merge(sales,features, on = ['Store','Date', 'IsHoliday'], how = 'left')
total = pd.merge(total,stores, on = ['Store'], how = 'left')

In [5]:
#fill NAs with 0
total = total.fillna(0)

In [6]:
#encode column 'Type' as numbers
types_encoded, types = total['Type'].factorize()
total['Type'] = types_encoded

In [7]:
#change temperature to degrees celsius
total['Temperature'] = (total['Temperature'] - 32) * 5/9

In [8]:
#convert IsHoliday column from boolean to int
total['IsHoliday'] = total['IsHoliday'] * 1

In [9]:
total.head(15)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-05-02,24924.50,0,5.727778,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,0,151315
1,1,1,2010-12-02,46039.49,1,3.616667,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,0,151315
2,1,1,2010-02-19,41595.55,0,4.405556,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,0,151315
3,1,1,2010-02-26,19403.54,0,8.127778,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,0,151315
4,1,1,2010-05-03,21827.90,0,8.055556,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,0,151315
5,1,1,2010-12-03,21043.39,0,14.327778,2.667,0.0,0.0,0.0,0.0,0.0,211.380643,8.106,0,151315
6,1,1,2010-03-19,22136.64,0,12.544444,2.720,0.0,0.0,0.0,0.0,0.0,211.215635,8.106,0,151315
7,1,1,2010-03-26,26229.21,0,10.805556,2.732,0.0,0.0,0.0,0.0,0.0,211.018042,8.106,0,151315
8,1,1,2010-02-04,57258.43,0,16.816667,2.719,0.0,0.0,0.0,0.0,0.0,210.820450,7.808,0,151315
9,1,1,2010-09-04,42960.91,0,18.811111,2.770,0.0,0.0,0.0,0.0,0.0,210.622857,7.808,0,151315
